<a href="https://colab.research.google.com/github/exkuretrol/concat-teams-mp4-and-generate-subtitles/blob/main/concat_mp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://downloads.rclone.org/v1.62.2/rclone-v1.62.2-linux-amd64.deb &> /dev/null
!apt install ./rclone-v1.62.2-linux-amd64.deb &> /dev/null
!sudo mkdir /content/onedrive
!sudo apt install fuse3 &> /dev/null

In [2]:
!rclone config

2023/04/17 06:18:35 NOTICE: Config file "/root/.config/rclone/rclone.conf" not found - using defaults
No remotes found, make a new one?
n) New remote
s) Set configuration password
q) Quit config
n/s/q> n

Enter name for new remote.
name> onedrive

Option Storage.
Type of storage to configure.
Choose a number from below, or type in your own value.
 1 / 1Fichier
   \ (fichier)
 2 / Akamai NetStorage
   \ (netstorage)
 3 / Alias for an existing remote
   \ (alias)
 4 / Amazon Drive
   \ (amazon cloud drive)
 5 / Amazon S3 Compliant Storage Providers including AWS, Alibaba, Ceph, China Mobile, Cloudflare, ArvanCloud, DigitalOcean, Dreamhost, Huawei OBS, IBM COS, IDrive e2, IONOS Cloud, Liara, Lyve Cloud, Minio, Netease, RackCorp, Scaleway, SeaweedFS, StackPath, Storj, Tencent COS, Qiniu and Wasabi
   \ (s3)
 6 / Backblaze B2
   \ (b2)
 7 / Better checksums for other remotes
   \ (hasher)
 8 / Box
   \ (box)
 9 / Cache a remote
   \ (cache)
10 / Citrix Sharefile
   \ (sharefile)
11 / Combin

In [3]:
!nohup rclone --vfs-cache-mode writes mount onedrive: /content/onedrive &

nohup: appending output to 'nohup.out'


In [4]:
from pathlib import Path
import datetime as dt
import tempfile
import re
import subprocess
import os

In [5]:
def concat_mp4(school_opening_day: str, recordings_dir: Path):
    course_name = recordings_dir.name
    first_week_of_course = ""
    
    first_run = True
    
    def get_datetime(filename: str):
        pattern = r'\d{8}_\d{6}'
        datetime_str = re.search(pattern, filename).group(0)
        return dt.datetime.strptime(datetime_str, "%Y%m%d_%H%M%S")
    
    p = re.compile(r'^.*\d{8}_\d{6}.*$')
    recordings = [ str(i) for i in recordings_dir.glob("*.mp4") if p.match(str(i))]
    sorted_recordings = sorted(recordings, key = get_datetime)
    sorted_recordings = [ Path(i) for i in sorted_recordings ]
    
    group_recordings = {}
    
    for recording in sorted_recordings:
        if (recording.is_file() and recording.suffix== '.mp4'):
            file = recording
            datetime_obj = get_datetime(file.stem)
    
            if (first_run):
                first_run = False
                course_weekday = datetime_obj.weekday()
                first_week_of_course = school_opening_day + dt.timedelta(days = course_weekday)
            
            diff = datetime_obj - first_week_of_course
            nth_week = (diff.days / 7) + 1
            if (nth_week.is_integer()):
                nth_week = int(nth_week)
            else:
                nth_week = round(nth_week, 1)
        
            if nth_week not in group_recordings:
                group_recordings[nth_week] = []
            group_recordings[nth_week].append(recording)
    
    for group_id, values in group_recordings.items():
        print(f"週次 {group_id}")
        for value in values:
            print(value.stem)
    
    response = input("\n上述的分類的週次都正確嗎? (yes/no) ")
    if response in ("yes", "no"):
        if response == "yes":
            for group_id, values in group_recordings.items():
                
                output_file = course_name + "-W" + str(group_id) + ".mp4"
                output_file = Path(recordings_dir / output_file)
                if len(values) > 1:
                    tmp = tempfile.NamedTemporaryFile(delete = False)
                    
                    for input_file in values:
                        line = f'file \'{input_file}\'\n'
                        tmp.write(line.encode())
                    tmp.close()
    
                    command = f'ffmpeg -f concat -safe 0 -i {tmp.name} -c copy {output_file}'
    
                    process = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                    
                    if process.returncode == 0:
                        print(f'Successfully concatenated the videos and saved them as {output_file}.')
                    else:
                        print(f'An error occurred during video concatenation. Error details: {process.stderr.decode("utf-8")}')
    
                    os.unlink(tmp.name)
                else:
                    print(f'Rename video filename as {output_file}.')
                    values[0].rename(output_file)
        else:
            print("abort")
            

In [6]:
#@title Default title text { run: "auto", display-mode: "form" }

#@markdown 開學日期
school_opening_day = "2023-02-13" #@param {type:"date"}
school_opening_day = dt.datetime.strptime(school_opening_day, "%Y-%m-%d")

#@markdown 目標資料夾路徑
recordings_dir = "/content/onedrive/\u9298\u50B3\u5927\u5B78/\u8AB2\u7A0B\u9304\u5F71/\u7DB2\u7AD9\u4F3A\u670D\u5668\u6982\u8AD6" #@param {type:"string"}
recordings_dir = Path(recordings_dir)


110 第一學期 2021/09/13

111 第一學期 2022/09/05

111 第二學期 2023/02/13


In [7]:
concat_mp4(school_opening_day, recordings_dir)

週次 1
「General」中的會議 -20230213_160242-Meeting Recording
週次 1.7
「General」中的會議 -20230218_155808-Meeting Recording
週次 4
「General」中的會議 -20230306_155658-Meeting Recording
週次 5
「General」中的會議 -20230313_155605-Meeting Recording
週次 6
「General」中的會議 -20230320_155339-Meeting Recording
週次 7
在 一般 中的會議-20230327_155842-Meeting Recording

上述的分類的週次都正確嗎? (yes/no) yes
Rename video filename as /content/onedrive/銘傳大學/課程錄影/網站伺服器概論/網站伺服器概論-W1.mp4.
Rename video filename as /content/onedrive/銘傳大學/課程錄影/網站伺服器概論/網站伺服器概論-W1.7.mp4.
Rename video filename as /content/onedrive/銘傳大學/課程錄影/網站伺服器概論/網站伺服器概論-W4.mp4.
Rename video filename as /content/onedrive/銘傳大學/課程錄影/網站伺服器概論/網站伺服器概論-W5.mp4.
Rename video filename as /content/onedrive/銘傳大學/課程錄影/網站伺服器概論/網站伺服器概論-W6.mp4.
Rename video filename as /content/onedrive/銘傳大學/課程錄影/網站伺服器概論/網站伺服器概論-W7.mp4.
